# Imports and initialization of general parameters
***

In [162]:
from helpers.pareto_fairness import compute_pareto_metrics
from config.info import AGES, RACES, GENDERS, COMBS_BASELINE, COMBS_FOULDS, COMBS_MARTINEZ
from visualization.subgroup_distribution import plot_dist
from dataprocess.dataloader import load_data
from dataprocess.dataclass import Data
from config.get_args import get_args
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from lightning import seed_everything
from plotly.subplots import make_subplots
from helpers.init import init
import numpy as np
import argparse
import pandas as pd
import random
import torch.nn as nn
import torch
import torchmetrics as tm
import pickle

loss_fct = nn.CrossEntropyLoss()
protected_atts = ['age_', 'race_', 'gender_']


# Auto reload part
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
# args
args = argparse.Namespace()
args.model = 'Baseline' 
args.ID = 1010
args.add_protected_attributes = 1
args.allow_wandb = False
args.job_id = 9
args = get_args(args = args)
args.fc_latent_size = [20, 20]

# Load the data, the model, wandb and the trainer
seed_everything(args.seed)
data, model, _, _ = init(**vars(args))

Global seed set to 194825
/home/gav061/Documents/Intersectional-Fairness-on-Histopathological-images-classification/dataprocess/dataloader.py:34: UserWarning:

The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400430266/work/aten/src/ATen/NestedTensorImpl.cpp:178.)



[ --------------- GENERAL --------------- ]
File name : cancer_classification_luad_lusc_PM.csv
Total nb of data : 800
Nb of features : 771
Split ratio : 0.2
Validation set : 1

Train set size : 501
Validation set size : 151
Test set size : 148



/home/gav061/.conda/envs/pdm/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning:

The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/gav061/.conda/envs/pdm/lib/python3.11/site-pac ...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gav061/.conda/envs/pdm/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning:

Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install li

In [170]:
data.TestSet.references[['age_', 'race_', 'gender_']].value_counts()

age_  race_  gender_
7     1      1          32
6     1      1          31
7     1      0          25
6     1      0          22
5     1      1          12
             0           8
8     1      1           7
             0           4
4     1      0           3
             1           3
3     1      0           1
Name: count, dtype: int64

# Compare the results
***

In [147]:
# Initialization
task = 'cancer_classification'
cancer = 'luad_lusc_PM'

# Get the Baseline results 
ID_b = 1000
preds_path = f'results/preds/run_{ID_b}/add_protected_atts_1/Baseline/{task}/{cancer}/best_results.pkl'
results_all_b = pd.read_pickle(preds_path)
results_b = results_all_b[results_all_b.set == 'test']

# Get the Martinez results 
ID_m = 1000
preds_path = f'results/preds/run_{ID_m}/add_protected_atts_1/Foulds/{task}/{cancer}/lambda_0.01/DF_pos/best_results.pkl'
results_all_m = pd.read_pickle(preds_path)
results_m = results_all_m[results_all_m.set == 'test']

In [161]:
atts = ['age_', 'race_', 'gender_']
np.unique(results_m[atts].values, axis = 0)

array([[3, 1, 0],
       [4, 1, 0],
       [4, 1, 1],
       [5, 1, 0],
       [5, 1, 1],
       [6, 1, 0],
       [6, 1, 1],
       [7, 1, 0],
       [7, 1, 1],
       [8, 1, 0],
       [8, 1, 1]])

In [148]:
def compute_density_and_metrics(results, results_all):
    # Initialization
    results = results.astype({'age_' : 'int32', 
                            'race_' : 'int32', 
                            'gender_' : 'int32', 
                            'pred' : 'int32', 
                            'label' : 'int32'})
    atts = ['age_', 'race_', 'gender_']
    subgroups = np.unique(results[atts].values, axis = 0)
    df = pd.DataFrame(subgroups, columns = atts)
    df['F1-score_0'] = 0.0
    df['F1-score_1'] = 0.0
    df['density'] = 0

    # Loop on all the subgroups
    for idx, sg in enumerate(subgroups):
        cond = (results.age_ == sg[0]) & (results.race_ == sg[1]) & (results.gender_ == sg[2])
        sub_df = results[cond]
        
        # Compute the f1-score
        f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(sub_df.pred.values), torch.Tensor(sub_df.label.values))
        df.loc[idx, 'F1-score_0'] = f1_scores[0].item()
        df.loc[idx, 'F1-score_1'] = f1_scores[1].item()
        
        # Density
        df.loc[idx, 'density'] = len(sub_df)
    
    mmpf_metrics = compute_pareto_metrics(results_all, protected_atts)
        
    # Modifications of the refs 
    df['age_str'] = df.age_.astype(str)
    df['race_str'] = df.apply(lambda row : RACES['str'][row['race_']], axis = 1)
    df['gender_str'] = df.apply(lambda row : GENDERS['str'][row['gender_']], axis = 1)
    return df, mmpf_metrics

In [150]:
# Get the metrics
df_b, b = compute_density_and_metrics(results_b, results_all_b)
df_m, m = compute_density_and_metrics(results_m, results_all_m)

# Pivot tables
d = pd.pivot_table(df_b[df_b.gender_ == 1][['age_str', 'race_str', 'density']], index = 'age_str', columns = 'race_str')
density = pd.pivot_table(df_b[df_b.race_ == 1][['age_str', 'gender_str', 'density']], index = 'age_str', columns = 'gender_str')
baseline_0 = pd.pivot_table(df_b[df_b.race_ == 1][['age_str', 'gender_str', 'F1-score_0']], index = 'age_str', columns = 'gender_str')
martinez_0 = pd.pivot_table(df_m[df_m.race_ == 1][['age_str', 'gender_str', 'F1-score_0']], index = 'age_str', columns = 'gender_str')
baseline_1 = pd.pivot_table(df_b[df_b.race_ == 1][['age_str', 'gender_str', 'F1-score_1']], index = 'age_str', columns = 'gender_str')
martinez_1 = pd.pivot_table(df_m[df_m.race_ == 1][['age_str', 'gender_str', 'F1-score_1']], index = 'age_str', columns = 'gender_str')

In [160]:
d

,density
race_str,WHITE
age_str,
4,3.0
5,12.0
6,31.0
7,32.0
8,7.0


In [152]:
density

density      
gender_str  FEMALE  MALE
age_str                 
3              1.0   NaN
4              3.0   3.0
5              8.0  12.0
6             22.0  31.0
7             25.0  32.0
8              4.0   7.0

In [153]:
baseline_0

F1-score_0          
gender_str     FEMALE      MALE
age_str                        
3            1.000000       NaN
4            0.500000  1.000000
5            0.857143  0.800000
6            0.969697  0.782609
7            0.944444  1.000000
8            1.000000  0.666667

In [158]:
martinez_0

F1-score_0     
gender_str     FEMALE MALE
age_str                   
3            1.000000  NaN
4            0.500000  1.0
5            0.923077  0.8
6            0.969697  0.7
7            0.972973  1.0
8            1.000000  0.4

In [154]:
baseline_1

F1-score_1          
gender_str     FEMALE      MALE
age_str                        
3            0.000000       NaN
4            0.000000  1.000000
5            0.000000  0.857143
6            0.909091  0.871795
7            0.857143  1.000000
8            1.000000  0.750000

In [159]:
martinez_1

F1-score_1          
gender_str     FEMALE      MALE
age_str                        
3            0.000000       NaN
4            0.000000  1.000000
5            0.666667  0.857143
6            0.909091  0.857143
7            0.923077  1.000000
8            1.000000  0.666667

In [157]:
density

density      
gender_str  FEMALE  MALE
age_str                 
3              1.0   NaN
4              3.0   3.0
5              8.0  12.0
6             22.0  31.0
7             25.0  32.0
8              4.0   7.0

In [ ]:
# Build the subplot
fig = make_subplots(2, 3, specs=[[{'r':0.02}, {'l':0.02}, {'l':0.02}]], #horizontal_spacing = 0.1,
                    subplot_titles=('Distribution of the data', 'Accuracy (Unfair scenario)', 'Accuracy (Fair scenario)', '', '', ''))

# Add the heatmaps
fig.add_trace(go.Heatmap(x = df_dist.columns, 
                         y = df_dist.index, 
                         z = df_dist, 
                         coloraxis = 'coloraxis', texttemplate = '%{z:.2f}'), 1, 1)
fig.add_trace(go.Heatmap(x = df_unfair.columns, 
                         y = df_unfair.index, 
                         z = df_unfair, 
                         coloraxis = 'coloraxis2', texttemplate = '%{z:.2f}'), 1, 2)
fig.add_trace(go.Heatmap(x = df_fair.columns,
                         y = df_fair.index, 
                         z = df_fair, 
                         coloraxis = 'coloraxis2', texttemplate = '%{z:.2f}'), 1, 3)

# Update the figures
fig.update_layout(height = 550, width = 1500,
                  template = 'none',
                  yaxis = {"title": 'att_2'},
                  yaxis2 = {"title": 'att_2'},
                  yaxis3 = {"title": 'att_2'},
                  xaxis = {"title": 'att_1'},
                  xaxis2 = {"title": 'att_1'},
                  xaxis3 = {"title": 'att_1'},
                  coloraxis = dict(colorscale='ice', colorbar_x=0.27, colorbar_thickness=23, 
                                   colorbar_title = 'Nb of data ', colorbar_title_side = 'right'),
                  coloraxis2 = dict(colorscale='RdYlGn', colorbar_x=1, colorbar_thickness=23, 
                                    colorbar_title = 'Accuracy', colorbar_title_side = 'right'))
fig.update_layout(xaxis1 = dict(tickvals=range_atts[0]),
                  yaxis1 = dict(tickvals=range_atts[1]),
                  xaxis2 = dict(tickvals=range_atts[0]),
                  yaxis2 = dict(tickvals=range_atts[1]),
                  xaxis3 = dict(tickvals=range_atts[0]),
                  yaxis3 = dict(tickvals=range_atts[1]))
fig.write_image("images/metric_1.eps")
fig.show()

# Baseline
***

In [109]:
def keep_best(df100 : pd.DataFrame, df1000 : pd.DataFrame):
    # Check the match between the cancer
    cs = [c for c in df100.cancer.to_list() if c in df1000.cancer.to_list()]
    df100.set_index('cancer', drop = False, inplace = True)
    df1000.set_index('cancer', drop = False, inplace = True)
    df = df100[df100.cancer.isin(cs)].copy()
    df['ID'] = 100
    
    # Check the best MMPF_size value between df100 and df1000, and add it to a new df
    for c in cs:
        
        # Compare the values 
        v1 = df100[df100.cancer == c]['MMPF_size'][0]
        v2 = df1000[df1000.cancer == c]['MMPF_size'][0]
        if v2 <= v1:
            df.loc[c, 'Accuracy'] = df1000[df1000.cancer == c]['Accuracy'][0]
            df.loc[c, 'F1-score_0'] = df1000[df1000.cancer == c]['F1-score_0'][0]
            df.loc[c, 'F1-score_1'] = df1000[df1000.cancer == c]['F1-score_1'][0]
            df.loc[c, 'MMPF_size'] = df1000[df1000.cancer == c]['MMPF_size'][0]
            df.loc[c, 'ID'] = 1000
    return df

def select_best_test(df_val, df100, df1000):
    # Copy the df
    cs = [c for c in df100.cancer.to_list() if c in df1000.cancer.to_list()]
    df100.set_index('cancer', drop = False, inplace = True)
    df1000.set_index('cancer', drop = False, inplace = True)
    df = df_val.copy()

    # Select the best inside the test df depending of the df val
    for c in cs:
        
        # Check which ID is the best validation MMPF value
        if df.loc[c, 'ID'] == 100:
            df.loc[c, 'Accuracy'] = df100[df100.cancer == c]['Accuracy'][0]
            df.loc[c, 'F1-score_0'] = df100[df100.cancer == c]['F1-score_0'][0]
            df.loc[c, 'F1-score_1'] = df100[df100.cancer == c]['F1-score_1'][0]
            df.loc[c, 'MMPF_size'] = df100[df100.cancer == c]['MMPF_size'][0]
        else:
            df.loc[c, 'Accuracy'] = df1000[df1000.cancer == c]['Accuracy'][0]
            df.loc[c, 'F1-score_0'] = df1000[df1000.cancer == c]['F1-score_0'][0]
            df.loc[c, 'F1-score_1'] = df1000[df1000.cancer == c]['F1-score_1'][0]
            df.loc[c, 'MMPF_size'] = df1000[df1000.cancer == c]['MMPF_size'][0]
            
    return df

In [110]:
def plot_results_baseline(df : pd.DataFrame, filename : str):
    # Initialization of the plot
    task_cc = 'cancer_classification'
    task_td = 'tumor_detection'
    fig = make_subplots(rows = 2, cols = 1, vertical_spacing = 0.2,
                        subplot_titles = ('Tumor detection tasks', 'Cancer classification tasks'))

    # Traces
    cc = px.bar(df[df.task == task_cc], x = 'cancer', y = ['Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size'],
                color_discrete_sequence = ['rgb(170, 220, 225)', 'rgb(80, 135, 180)', 'rgb(50, 100, 170)', 'rgb(21, 21, 45)'], text_auto = '.3f')
    td = px.bar(df[df.task == task_td], x = 'cancer', y = ['Accuracy', 'F1-score_1', 'MMPF_size'],
                color_discrete_sequence = ['rgb(170, 220, 225)', 'rgb(50, 100, 170)', 'rgb(21, 21, 45)'], text_auto = '.3f').update_traces(showlegend = False)

    # Update the figure
    for t in range(len(cc['data'])): fig.add_trace(cc['data'][t], row = 2, col = 1)
    for t in range(len(td['data'])): fig.add_trace(td['data'][t], row = 1, col = 1)
    fig.update_layout(height = 700, width = 1200, 
                    template = 'none',
                    xaxis = {'title': 'Cancer'},
                    yaxis = {'title': 'Values [no unit]'},
                    xaxis2 = {'title': 'Cancer'},
                    yaxis2 = {'title': 'Values [no unit]'})
    fig.update_layout(barmode = 'group', bargap = 0.3, bargroupgap = 0.1)
    fig.write_image(filename)
    fig.show()
    
def plot_results_others(df : pd.DataFrame, filename : str):
    # Initialization of the plot
    task_cc = 'cancer_classification'

    # Traces
    fig = px.bar(df[df.task == task_cc], x = 'cancer', y = ['Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size'],
                color_discrete_sequence = ['rgb(170, 220, 225)', 'rgb(80, 135, 180)', 'rgb(50, 100, 170)', 'rgb(21, 21, 45)'], text_auto = '.3f')

    # Update the figure
    # for t in range(len(cc['data'])): fig.add_trace(cc['data'][t], row = 1, col = 1)
    fig.update_layout(height = 400, width = 1200, 
                      title = 'Cancer classification tasks',
                    template = 'none',
                    xaxis = {'title': 'Cancer'},
                    yaxis = {'title': 'Values [no unit]'})
    fig.update_layout(barmode = 'group', bargap = 0.3, bargroupgap = 0.1)
    fig.write_image(filename)
    fig.show()

In [137]:
def check_results_baseline(task : str, 
                           cancer : str, 
                           df : pd.DataFrame,
                           set_ : str, ID : int):
    # Initialization
    sub_dict = {'task' : [task], 'cancer' : [cancer]}
    
    # Extract the results pkl files
    preds_path = f'results/preds/run_{ID}/add_protected_atts_1/Baseline'
    results = pd.read_pickle(preds_path + f'/{task}/{cancer}/best_results.pkl')
    
    # Compute the metrics
    # Accuracy 
    sub_dict['Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    
    # F1-score
    # sub_dict['F1-score'] = tm.classification.BinaryF1Score()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(results.pred), torch.Tensor(results.label))
    sub_dict['F1-score_0'] = f1_scores[0].item()
    sub_dict['F1-score_1'] = f1_scores[1].item()
    
    # MMPF_size
    mmpf_metrics = compute_pareto_metrics(results, protected_atts)
    sub_dict['MMPF_size'] = [mmpf_metrics[f'{set_}MMPF_size_set_2']]
    
    # Return the updated data frame
    if df.empty: df = pd.DataFrame(data = sub_dict)
    else: df = pd.concat([df, pd.DataFrame(data = sub_dict)])
    return df

# Initialization
cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
df_baseline_100 = pd.DataFrame(columns = cols)
df_baseline_1000 = pd.DataFrame(columns = cols)
df_baseline_100_test = pd.DataFrame(columns = cols)
df_baseline_1000_test = pd.DataFrame(columns = cols)

# Loop on the combinations for baseline
for comb in COMBS_BASELINE[2:]:
    # Track
    print(comb)
    
    # Extract combinations
    task = comb[0]
    cancer = comb[1]
    
    # Get the metrics - validation
    df_baseline_100 = check_results_baseline(task, cancer, df_baseline_100, 'val_', 100)
    df_baseline_1000 = check_results_baseline(task, cancer, df_baseline_1000, 'val_', 1000)
    
    # Get the metrics - test
    df_baseline_100_test = check_results_baseline(task, cancer, df_baseline_100_test, 'test_', 100)
    df_baseline_1000_test = check_results_baseline(task, cancer, df_baseline_1000_test, 'test_', 1000)

# Plot
df_b = keep_best(df_baseline_100, df_baseline_1000)
df_b_test = select_best_test(df_b, df_baseline_100_test, df_baseline_1000_test)
plot_results_baseline(df_b_test, 'images/results_baseline.eps')

['cancer_classification', 'kich_kirc_FS']
['cancer_classification', 'kich_kirc_PM']
['cancer_classification', 'kich_kirp_FS']
['cancer_classification', 'kich_kirp_PM']
['cancer_classification', 'kirc_kirp_FS']
['cancer_classification', 'kirc_kirp_PM']
['cancer_classification', 'luad_lusc_FS']
['cancer_classification', 'luad_lusc_PM']
['tumor_detection', 'brca']
['tumor_detection', 'coad']
['tumor_detection', 'kich']
['tumor_detection', 'kirc']
['tumor_detection', 'kirp']
['tumor_detection', 'luad']
['tumor_detection', 'lusc']
['tumor_detection', 'read']


/tmp/ipykernel_28248/2845827234.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as label

In [144]:
df_b

,task,cancer,Accuracy,F1-score_0,F1-score_1,MMPF_size,ID
cancer,,,,,,,
kich_kirc_FS,cancer_classification,kich_kirc_FS,0.982929,0.929889,0.990281,0.428646,100
kich_kirc_PM,cancer_classification,kich_kirc_PM,0.969479,0.918660,0.981215,0.365892,1000
kich_kirp_FS,cancer_classification,kich_kirp_FS,0.985222,0.976923,0.989130,0.379928,100
kich_kirp_PM,cancer_classification,kich_kirp_PM,0.993671,0.989691,0.995434,0.413262,100
kirc_kirp_FS,cancer_classification,kirc_kirp_FS,0.987251,0.991786,0.971530,0.344152,1000
kirc_kirp_PM,cancer_classification,kirc_kirp_PM,0.994126,0.995662,0.990909,0.356744,1000
luad_lusc_FS,cancer_classification,luad_lusc_FS,0.921099,0.924384,0.917516,0.471156,100
luad_lusc_PM,cancer_classification,luad_lusc_PM,0.942500,0.947368,0.936639,0.436482,1000
brca,tumor_detection,brca,0.991150,0.981187,0.994215,0.328646,1000


# Foulds
***

In [139]:
def check_results_foulds(task : str, 
                           cancer : str, 
                           lambda_ : float,
                           pt_method : str,
                           df : pd.DataFrame,
                           set_ : str, ID : int):
    # Initialization
    sub_dict = {'task' : [task], 'cancer' : [cancer], 'lambda_' : [lambda_], 'pt_method' : [pt_method]}
    
    # Extract the results pkl files
    preds_path = f'results/preds/run_{ID}/add_protected_atts_1/Foulds'
    results = pd.read_pickle(preds_path + f'/{task}/{cancer}/lambda_{lambda_}/{pt_method}/best_results.pkl')
    
    # Compute the metrics
    # Accuracy 
    sub_dict['Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    
    # F1-score
    # sub_dict['F1-score'] = tm.classification.BinaryF1Score()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(results.pred), torch.Tensor(results.label))
    sub_dict['F1-score_0'] = f1_scores[0].item()
    sub_dict['F1-score_1'] = f1_scores[1].item()
    
    # MMPF_size
    mmpf_metrics = compute_pareto_metrics(results, protected_atts)
    sub_dict['MMPF_size'] = [mmpf_metrics[f'{set_}MMPF_size_set_2']]
    
    # Return the updated data frame
    if df.empty: df = pd.DataFrame(data = sub_dict)
    else: df = pd.concat([df, pd.DataFrame(data = sub_dict)])
    return df


def select_hyper(df_foulds):
    # Initialization
    df_foulds.reset_index(inplace = True, drop = True)
    cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
    df_foulds_best = pd.DataFrame(columns = cols)

    # Loop on the combinations for baseline
    for comb in COMBS_BASELINE[2:10]:
        
        # Extract combinations
        task = comb[0]
        cancer = comb[1]
        cond = (df_foulds.task == task) & (df_foulds.cancer == cancer)
        
        # Get the metrics from the models with the best hyperparameters
        # Get hyperparameters
        best_idx = df_foulds[cond].MMPF_size.argmin()
        sub_dict = {'task' : [task], 'cancer' : [cancer]}
        sub_dict['Accuracy'] = df_foulds[cond].iloc[best_idx]['Accuracy']
        sub_dict['F1-score_0'] = df_foulds[cond].iloc[best_idx]['F1-score_0']
        sub_dict['F1-score_1'] = df_foulds[cond].iloc[best_idx]['F1-score_1']
        sub_dict['MMPF_size'] = df_foulds[cond].iloc[best_idx]['MMPF_size']
        
        # Update the dataframe
        if df_foulds_best.empty: df_foulds_best = pd.DataFrame(data = sub_dict)
        else: df_foulds_best = pd.concat([df_foulds_best, pd.DataFrame(data = sub_dict)])
    
    return df_foulds_best


# Initialization
cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
df_foulds_100 = pd.DataFrame(columns = cols)
df_foulds_1000 = pd.DataFrame(columns = cols)
df_foulds_100_test = pd.DataFrame(columns = cols)
df_foulds_1000_test = pd.DataFrame(columns = cols)

# Loop on the combinations for baseline
for comb in COMBS_FOULDS[6:30]:
    # Track
    print(comb)
    
    # Extract combinations
    task = comb[0]
    cancer = comb[1]
    l = comb[2]
    pt_method = comb[3]
    
    # Get the metrics
    df_foulds_100 = check_results_foulds(task, cancer, l, pt_method, df_foulds_100, 'val_', 100)
    df_foulds_1000 = check_results_foulds(task, cancer, l, pt_method, df_foulds_1000, 'val_', 1000)
    
    # Get the metrics
    df_foulds_100_test = check_results_foulds(task, cancer, l, pt_method, df_foulds_100_test, 'test_', 100)
    df_foulds_1000_test = check_results_foulds(task, cancer, l, pt_method, df_foulds_1000_test, 'test_', 1000)
    
# Select hyperparameters
df_foulds_100_ = select_hyper(df_foulds_100)
df_foulds_1000_ = select_hyper(df_foulds_1000)
df_foulds_100_test_ = select_hyper(df_foulds_100_test)
df_foulds_1000_test_ = select_hyper(df_foulds_1000_test)

df_f = keep_best(df_foulds_100_, df_foulds_1000_)
df_f_test = select_best_test(df_f, df_foulds_100_test_, df_foulds_1000_test_)
plot_results_others(df_f_test, 'images/results_foulds.eps')

['cancer_classification', 'kich_kirc_FS', 0.01, 'DF_pos']
['cancer_classification', 'kich_kirc_FS', 0.01, 'DF_sum']
['cancer_classification', 'kich_kirc_FS', 0.01, 'DF_max']
['cancer_classification', 'kich_kirc_PM', 0.01, 'DF_pos']
['cancer_classification', 'kich_kirc_PM', 0.01, 'DF_sum']
['cancer_classification', 'kich_kirc_PM', 0.01, 'DF_max']
['cancer_classification', 'kich_kirp_FS', 0.01, 'DF_pos']
['cancer_classification', 'kich_kirp_FS', 0.01, 'DF_sum']
['cancer_classification', 'kich_kirp_FS', 0.01, 'DF_max']
['cancer_classification', 'kich_kirp_PM', 0.01, 'DF_pos']
['cancer_classification', 'kich_kirp_PM', 0.01, 'DF_sum']
['cancer_classification', 'kich_kirp_PM', 0.01, 'DF_max']
['cancer_classification', 'kirc_kirp_FS', 0.01, 'DF_pos']
['cancer_classification', 'kirc_kirp_FS', 0.01, 'DF_sum']
['cancer_classification', 'kirc_kirp_FS', 0.01, 'DF_max']
['cancer_classification', 'kirc_kirp_PM', 0.01, 'DF_pos']
['cancer_classification', 'kirc_kirp_PM', 0.01, 'DF_sum']
['cancer_class

/tmp/ipykernel_28248/2845827234.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as label

In [145]:
df_f

,task,cancer,Accuracy,F1-score_0,F1-score_1,MMPF_size,ID
cancer,,,,,,,
kich_kirc_FS,cancer_classification,kich_kirc_FS,0.967655,0.878378,0.981347,0.420411,1000
kich_kirc_PM,cancer_classification,kich_kirc_PM,0.962298,0.901408,0.976693,0.390165,1000
kich_kirp_FS,cancer_classification,kich_kirp_FS,0.982759,0.973978,0.987109,0.313360,1000
kich_kirp_PM,cancer_classification,kich_kirp_PM,0.990506,0.984615,0.993135,0.413262,100
kirc_kirp_FS,cancer_classification,kirc_kirp_FS,0.987251,0.991803,0.971326,0.329218,1000
kirc_kirp_PM,cancer_classification,kirc_kirp_PM,0.989721,0.992416,0.984055,0.356747,1000
luad_lusc_FS,cancer_classification,luad_lusc_FS,0.891844,0.893170,0.890485,0.460775,1000
luad_lusc_PM,cancer_classification,luad_lusc_PM,0.951250,0.955017,0.946794,0.430336,1000


In [141]:
# FOR THE TABLES IN THE REPORT
reduced_df = df_foulds_1000.copy()
table = pd.pivot_table(reduced_df, values = 'MMPF_size', index = 'cancer', columns = ['pt_method'])
table
# print(table.to_latex())

pt_method,DF_max,DF_pos,DF_sum
cancer,,,
kich_kirc_FS,0.420411,0.443042,0.441798
kich_kirc_PM,0.423676,0.424372,0.390165
kich_kirp_FS,0.513258,0.513261,0.313360
kich_kirp_PM,0.813191,0.564594,0.813247
kirc_kirp_FS,0.332992,0.354926,0.329218
kirc_kirp_PM,0.411430,0.356751,0.356747
luad_lusc_FS,0.460775,0.481376,0.485634
luad_lusc_PM,0.434840,0.430336,0.435325


In [116]:
print(table.to_latex())

\begin{tabular}{lrrr}
\toprule
pt_method & DF_max & DF_pos & DF_sum \\
cancer &  &  &  \\
\midrule
kich_kirc_FS & 0.420411 & 0.443042 & 0.441798 \\
kich_kirc_PM & 0.365901 & 0.365892 & 0.365892 \\
kich_kirp_FS & 0.313262 & 0.313263 & 0.313262 \\
kich_kirp_PM & 0.313262 & 0.313262 & 0.313262 \\
kirc_kirp_FS & 0.313608 & 0.313294 & 0.313407 \\
kirc_kirp_PM & 0.313265 & 0.313265 & 0.313263 \\
luad_lusc_FS & 0.460775 & 0.481376 & 0.485634 \\
luad_lusc_PM & 0.434840 & 0.430336 & 0.435325 \\
\bottomrule
\end{tabular}



In [ ]:
# FOR THE TABLES IN THE REPORT
reduced_df = df_foulds[(df_foulds.task == t) & (df_foulds.cancer == c)]
table = pd.pivot_table(reduced_df, values = 'MMPF_size', index = 'pt_method', columns = ['lambda_'])
print(table.to_latex())

# Diana 
***

In [142]:
def check_results_diana(task : str, 
                           cancer : str, 
                           df : pd.DataFrame,
                           set_ : str, ID : int):
    # Initialization
    sub_dict = {'task' : [task], 'cancer' : [cancer]}
    
    # Extract the results pkl files
    preds_path = f'results/preds/run_{ID}/add_protected_atts_1/Diana'
    results = pd.read_pickle(preds_path + f'/{task}/{cancer}/results.pkl')
    
    # Compute the metrics
    # Accuracy 
    sub_dict['Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    
    # F1-score
    # sub_dict['F1-score'] = tm.classification.BinaryF1Score()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(results.pred), torch.Tensor(results.label))
    sub_dict['F1-score_0'] = f1_scores[0].item()
    sub_dict['F1-score_1'] = f1_scores[1].item()
    
    # MMPF_size
    mmpf_metrics = compute_pareto_metrics(results, protected_atts)
    sub_dict['MMPF_size'] = [mmpf_metrics[f'{set_}MMPF_size_set_2']]
    
    # Return the updated data frame
    if df.empty: df = pd.DataFrame(data = sub_dict)
    else: df = pd.concat([df, pd.DataFrame(data = sub_dict)])
    return df

# Initialization
cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
df_diana_100 = pd.DataFrame(columns = cols)
df_diana_1000 = pd.DataFrame(columns = cols)
df_diana_100_test = pd.DataFrame(columns = cols)
df_diana_1000_test = pd.DataFrame(columns = cols)

# Loop on the combinations for baseline
for comb in COMBS_BASELINE[2:10]:
    # Track
    print(comb)
    
    # Extract combinations
    task = comb[0]
    cancer = comb[1]
    
    # Get the metrics
    df_diana_100 = check_results_diana(task, cancer, df_diana_100, 'val_', 100)
    df_diana_1000 = check_results_diana(task, cancer, df_diana_1000, 'val_', 1000)
    
    # Get the metrics
    df_diana_100_test = check_results_diana(task, cancer, df_diana_100_test, 'test_', 100)
    df_diana_1000_test = check_results_diana(task, cancer, df_diana_1000_test, 'test_', 1000)

df_d = keep_best(df_diana_100, df_diana_1000)
df_d_test = select_best_test(df_d, df_diana_100_test, df_diana_1000_test)
plot_results_others(df_d_test, 'images/results_diana.eps')

['cancer_classification', 'kich_kirc_FS']
['cancer_classification', 'kich_kirc_PM']
['cancer_classification', 'kich_kirp_FS']
['cancer_classification', 'kich_kirp_PM']
['cancer_classification', 'kirc_kirp_FS']
['cancer_classification', 'kirc_kirp_PM']
['cancer_classification', 'luad_lusc_FS']
['cancer_classification', 'luad_lusc_PM']


/tmp/ipykernel_28248/2845827234.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as label

# Martinez
***

In [143]:
def check_results_martinez(task : str, 
                           cancer : str, 
                           df : pd.DataFrame,
                           set_ : str, ID : int):
    # Initialization
    sub_dict = {'task' : [task], 'cancer' : [cancer]}
    
    # Extract the results pkl files
    preds_path = f'results/preds/run_{ID}/add_protected_atts_1/Martinez'
    if ID == 100 : results = pd.read_pickle(preds_path + f'/{task}/{cancer}/alpha_0.5/results.pkl')
    else: results = pd.read_pickle(preds_path + f'/{task}/{cancer}/results.pkl')
    
    # Compute the metrics
    # Accuracy 
    sub_dict['Accuracy'] = tm.classification.BinaryAccuracy()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    
    # F1-score
    # sub_dict['F1-score'] = tm.classification.BinaryF1Score()(torch.Tensor(results.pred), torch.Tensor(results.label)).item()
    f1_scores = tm.classification.MulticlassF1Score(num_classes = 2, average = None)(torch.Tensor(results.pred), torch.Tensor(results.label))
    sub_dict['F1-score_0'] = f1_scores[0].item()
    sub_dict['F1-score_1'] = f1_scores[1].item()
    
    # MMPF_size
    mmpf_metrics = compute_pareto_metrics(results, protected_atts)
    sub_dict['MMPF_size'] = [mmpf_metrics[f'{set_}MMPF_size_set_2']]
    
    # Return the updated data frame
    if df.empty: df = pd.DataFrame(data = sub_dict)
    else: df = pd.concat([df, pd.DataFrame(data = sub_dict)])
    return df

# Initialization
cols = ['task', 'cancer', 'Accuracy', 'F1-score_0', 'F1-score_1', 'MMPF_size']
df_martinez_100 = pd.DataFrame(columns = cols)
df_martinez_1000 = pd.DataFrame(columns = cols)
df_martinez_100_test = pd.DataFrame(columns = cols)
df_martinez_1000_test = pd.DataFrame(columns = cols)

# Loop on the combinations for baseline
for comb in COMBS_BASELINE[2:10]:
    # Track
    print(comb)
    
    # Extract combinations
    task = comb[0]
    cancer = comb[1]
    
    # Get the metrics
    df_martinez_100 = check_results_martinez(task, cancer, df_martinez_100, 'val_', 100)
    df_martinez_1000 = check_results_martinez(task, cancer, df_martinez_1000, 'val_', 1000)

    # Get the metrics
    df_martinez_100_test = check_results_martinez(task, cancer, df_martinez_100_test, 'test_', 100)
    df_martinez_1000_test = check_results_martinez(task, cancer, df_martinez_1000_test, 'test_', 1000)
    
df_m = keep_best(df_martinez_100, df_martinez_1000)
df_m_test = select_best_test(df_m, df_martinez_100_test, df_martinez_1000_test)
plot_results_others(df_m_test, 'images/results_martinez.eps')

['cancer_classification', 'kich_kirc_FS']
['cancer_classification', 'kich_kirc_PM']
['cancer_classification', 'kich_kirp_FS']
['cancer_classification', 'kich_kirp_PM']
['cancer_classification', 'kirc_kirp_FS']
['cancer_classification', 'kirc_kirp_PM']
['cancer_classification', 'luad_lusc_FS']
['cancer_classification', 'luad_lusc_PM']


/tmp/ipykernel_28248/2845827234.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_28248/2845827234.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as label

In [121]:
df_m

,task,cancer,Accuracy,F1-score_0,F1-score_1,MMPF_size,ID
cancer,,,,,,,
kich_kirc_FS,cancer_classification,kich_kirc_FS,0.964061,0.856115,0.979466,0.413262,100
kich_kirc_PM,cancer_classification,kich_kirc_PM,0.953321,0.880734,0.970982,0.365893,1000
kich_kirp_FS,cancer_classification,kich_kirp_FS,0.960591,0.940741,0.970480,0.355066,1000
kich_kirp_PM,cancer_classification,kich_kirp_PM,0.987342,0.979592,0.990826,0.315064,1000
kirc_kirp_FS,cancer_classification,kirc_kirp_FS,0.984861,0.990291,0.965642,0.335001,100
kirc_kirp_PM,cancer_classification,kirc_kirp_PM,0.988253,0.991304,0.981900,0.350299,100
luad_lusc_FS,cancer_classification,luad_lusc_FS,0.890071,0.892734,0.887273,0.398368,100
luad_lusc_PM,cancer_classification,luad_lusc_PM,0.916250,0.922900,0.908345,0.438262,100
